# Monte Carlo Project

## Iacob Jessica, Mourre Grégoire

In this Jupter file, we will implement the different methods presented and described in https://github.com/gregoiremrr/Monte-Carlo-for-American-Options.


In [191]:
import numpy as np
from numpy.random import default_rng
from scipy.optimize import curve_fit
from tqdm import tqdm

# Methods for the lower bound

In [192]:
n = 10000
m = 12
rng = default_rng()

r, sigma, x, K, T = 0, .3, 100, 105, 1
dt = T/m

def g(x,t=0):
    return np.exp(-r*t*dt) * np.maximum(K-x,0)

def phi(x):
    # Canonical basis
    #return np.array([1, x, x**2, x**3], dtype=object)
    #return np.array([1, x, x**2, x**3, g(x)], dtype=object)
    #return np.array([1, x, x**2, x**3, x**4, g(x)], dtype=object)
    #return np.array([1, x, x**2, x**3, g(x), g(x)**2], dtype=object)
    return np.array([1, x, x**2, x**3, g(x), g(x)**2, g(x)**3], dtype=object)
    # Legendre's basis
    #return np.array([1, x, (3*x**2-1)/2, (5*x**3-3*x)/2], dtype=object)
    #return np.array([1, x, (3*x**2-1)/2, (5*x**3-3*x)/2, g(x)], dtype=object)
    #return np.array([1, x, (3*x**2-1)/2, (5*x**3-3*x)/2, (35*x**4-30*x**2+3)/8, g(x)], dtype=object)
    #return np.array([1, x, (3*x**2-1)/2, (5*x**3-3*x)/2, g(x), g(x)**2], dtype=object)
    #return np.array([1, x, (3*x**2-1)/2, (5*x**3-3*x)/2, g(x), g(x)**2, g(x)**3], dtype=object)
    # Hermite's basis
    #return np.array([1, x, x**2-1, x**3-3*x], dtype=object)
    #return np.array([1, x, x**2-1, x**3-3*x, g(x)], dtype=object)
    #return np.array([1, x, x**2-1, x**3-3*x, x**4-6*x**2+3, g(x)], dtype=object)
    #return np.array([1, x, x**2-1, x**3-3*x, g(x), g(x)**2], dtype=object)
    #return np.array([1, x, x**2-1, x**3-3*x, g(x), g(x)**2, g(x)**3], dtype=object)
    # Laguerre's basis
    #return np.array([1, 1-x, (x**2-4*x+2)/2, (-x**3+9*x**2-18*x+6)/6], dtype=object)
    #return np.array([1, 1-x, (x**2-4*x+2)/2, (-x**3+9*x**2-18*x+6)/6, g(x)], dtype=object)
    #return np.array([1, 1-x, (x**2-4*x+2)/2, (-x**3+9*x**2-18*x+6)/6, (x**4-16*x**3+72*x**2-96*x+24)/24, g(x)], dtype=object)
    #return np.array([1, 1-x, (x**2-4*x+2)/2, (-x**3+9*x**2-18*x+6)/6, g(x), g(x)**2], dtype=object)
    #return np.array([1, 1-x, (x**2-4*x+2)/2, (-x**3+9*x**2-18*x+6)/6, g(x), g(x)**2, g(x)**3], dtype=object)
l = len(phi(0))

def reg(x, *alphas):
    alpha = np.array(alphas)
    return np.sum(alpha * phi(x))

# Tsitsiklis & Van Roy's algorithm

In [193]:
# Step 1
B = rng.normal(size=m*n).reshape(m,n)
X = np.cumprod(np.concatenate([[x*np.ones(n)], 1 + r * dt + sigma * np.sqrt(dt) * B]), axis=0)

# Step 2
V = np.zeros(n*(m+1)).reshape(m+1,n)
V[-1,:] = g(X[-1,:],m)

# Step 3
alpha0 = np.zeros(l)
for i in range(m-1, 0, -1):
    alpha_star = curve_fit(reg, X[i,:], V[i+1,:], p0=alpha0)[0]
    _1 = g(X[i,:],i)
    _2 = reg(X[i,:], alpha_star)
    V[i,:] = _1 * (_1 >= _2) + _2 * (_1 < _2)

# Step 4
_1 = g(x)
_2 = np.mean(V[1,:])
V0 = _1 * (_1 >= _2) + _2 * (_1 < _2)

print("Estimator:", V0)

Estimator: 15.279211767481177


# Longstaff and Schwartz's algorithm
Approximation of the Value Functions (Steps 2 to 4 in Longstaff & Schwartz's algorithm)

In [194]:
# Step 2
V2 = np.zeros(n*(m+1)).reshape((m+1),n)
V2[-1,:] = g(X[-1,:],m)

alpha_tau = np.zeros(l*(m+1)).reshape((m+1),l)

# Step 3
alpha0 = np.zeros(l)
for i in range(m-1, 0, -1):
    alpha_tau[i,:] = curve_fit(reg, X[i,:], V2[i+1,:], p0=alpha0)[0]
    _1 = g(X[i,:],i)
    _2 = reg(X[i,:], alpha_tau[i,:])
    V2[i,:] = _1 * (_1 >= _2) + V2[(i+1),:] * (_1 < _2)

# Step 4 (not used)
_1 = g(x)
_2 = np.mean(V2[1,:])
V02 = np.mean(_1 * (_1 >= _2) + V2[1,:] * (_1 < _2))

Longstaff & Schwartz's algorithm (Step 5)

In [195]:
# Step 5
B2 = rng.normal(size=m*n).reshape(m,n)
X2 = np.cumprod(np.concatenate([[x*np.ones(n)], 1 + r * dt + sigma * np.sqrt(dt) * B2]), axis=0)

V3 = np.zeros(n*(m+1)).reshape((m+1),n)
V3[-1,:] = g(X2[-1,:],m)

for i in range(m-1, 0, -1):
    _1 = g(X2[i,:],i)
    _2 = reg(X2[i,:], alpha_tau[i,:])
    V3[i,:] = _1 * (_1 >= _2) + V3[(i+1),:] * (_1 < _2)

_1 = g(x)
_2 = np.mean(V3[1,:])
V03_ = _1 * (_1 >= _2) + V3[1,:] * (_1 < _2)

V03 = np.mean(V03_)
std = np.std(V03_)
conv_interval = V03 + np.array([-1,1]) * 1.96 * std / np.sqrt(n)
final_lowerbound = conv_interval[0]

print("Estimator:", V03)
print("Standard deviation:", std / np.sqrt(n))
print("Condidence interval 95%:", conv_interval)
print("Error:", 100 * 1.96 * std / (V03 * np.sqrt(n)), "%")

Estimator: 14.861717427527305
Standard deviation: 0.16009753836884097
Condidence interval 95%: [14.54792625 15.1755086 ]
Error: 2.11140587710082 %


# Martingales from Approximate Value Functions

In [196]:
# One trajectory (just to get the idea of the next reel method)

n2 = 1000

B_upperbound = rng.normal(size=m)
X_upperbound = np.cumprod(np.concatenate([[x], 1 + r * dt + sigma * np.sqrt(dt) * B_upperbound]), axis=0)
Normal = rng.normal(size=m*n2).reshape(m,n2)
M = np.zeros(m+1)
Mi = 0

for i in range(1, m):
    V_upperbound = max(g(X_upperbound[i],i), reg(X_upperbound[i], alpha_tau[i,:]))
    X_next = X_upperbound[i-1] * (1 + r * dt + sigma * np.sqrt(dt) * Normal[i-1,:])
    V_Ynext = np.maximum(g(X_next,i), reg(X_next, alpha_tau[i,:]))
    delta = V_upperbound - np.mean(V_Ynext)
    Mi += delta
    M[i] = Mi

X_next = X_upperbound[-2] * (1 + r * dt + sigma * np.sqrt(dt) * Normal[-1,:])
delta = g(X_upperbound[-1],m) - np.mean(g(X_next,m))
Mi += delta
M[-1] = Mi

V0_upperbound = np.max(g(X_upperbound,np.arange(0,m+1)) - M)
print(V0_upperbound)

14.45734083648952


In [197]:
# Monte-Carlo method for n trajectories

n2 = 1000

Normal = rng.normal(size=m*n*n2).reshape(m,n2,n)
M = np.zeros(n*(m+1)).reshape(m+1,n)
Mi = np.zeros(n)

for i in tqdm(range(1, m)):
    V_upperbound = np.maximum(g(X2[i,:],i), reg(X2[i,:], alpha_tau[i,:]))
    X_next = X2[i-1,:] * (1 + r * dt + sigma * np.sqrt(dt) * Normal[i-1,:,:])
    V_Ynext = np.maximum(g(X_next,i), reg(X_next, alpha_tau[i,:]))
    Mi += V_upperbound - np.mean(V_Ynext, axis=0)
    M[i,:] = Mi

X_next = X2[-2,:] * (1 + r * dt + sigma * np.sqrt(dt) * Normal[-1,:,:])
Mi += g(X2[-1,:],m) - np.mean(g(X_next,m), axis=0)
M[-1,:] = Mi

V0_upperbound_ = np.max(g(X2.T,np.arange(0,m+1)).T - M, axis=0)
V0_upperbound = np.mean(V0_upperbound_)
std_upperbound = np.std(V0_upperbound_)
conv_interval_upperbound = V0_upperbound + np.array([-1,1]) * 1.96 * std_upperbound / np.sqrt(n)
final_upperbound = conv_interval_upperbound[1]

print("Estimator:", V0_upperbound)
print("Standard deviation:", std_upperbound / np.sqrt(n))
print("Condidence interval 95%:", conv_interval_upperbound)
print("Error:", 100 * 1.96 * std_upperbound / (V0_upperbound * np.sqrt(n)), "%")

100%|██████████| 11/11 [00:04<00:00,  2.33it/s]

Estimator: 15.013743133245947
Standard deviation: 0.008913741323748284
Condidence interval 95%: [14.9962722  15.03121407]
Error: 0.11636627081929735 %


In [198]:
print("Confidence interval (lower & upper bounds):", [final_lowerbound, final_upperbound])

Confidence interval (lower & upper bounds): [14.547926252324377, 15.031214066240494]


# Martingales from Stopping Rules

In [199]:
# Step 1
n_subpaths = 500

Normal = rng.normal(size=m*m*n*n_subpaths).reshape(n,m,m,n_subpaths)

subpaths = np.zeros(m*(m+1)*n*n_subpaths).reshape(n,m,m+1,n_subpaths)
for k in tqdm(range(n)):
    for i in range(m):
        subpaths[k,i,:i+1,:] = np.repeat(X2[:i+1,k], n_subpaths).reshape(i+1,-1)
        subpaths[k,i,i+1:,:] = X2[i,k] * np.cumprod(1 + r * dt + sigma * np.sqrt(dt) * Normal[k,i,i:,:], axis=0)

subpaths_values = np.zeros(m*(m+1)*n*n_subpaths).reshape(n,m,m+1,n_subpaths)
for k in tqdm(range(n)):
    for i in range(m):
        subpaths_values[k,i,m,:] = g(subpaths[k,i,m,:],m)
        for j in range(m-1,i,-1):
            _1 = g(subpaths[k,i,j,:],j)
            _2 = reg(subpaths[k,i,j,:], alpha_tau[j,:])
            subpaths_values[k,i,j,:] =  _1 * (_1 >= _2) + subpaths_values[k,i,j+1,:] * (_1 < _2)

# approximation of E[ h_{\tau_{i+1}}(X_{\tau_{i+1}}) | X_i ] = V_iplus1[:,i]
V_iplus1 = np.zeros(n*m).reshape(n,m)
for i in range(m):
    V_iplus1[:,i] = np.mean(subpaths_values[:,i,i+1,:], axis = 1)

# approximation of E[ h_{\tau_i}(X_{\tau_i}) | X_i ] = V_i[:,i]
V_i = np.zeros(n*(m+1)).reshape(n,m+1)
for i in range(1,m):
    _1 = g(X2[i,:],i)
    _2 = reg(X2[i,:], alpha_tau[i,:])
    V_i[:,i] = _1 * (_1 >= _2) + V_iplus1[:,i] * (_1 < _2)

V_i[:,m] = g(X2[m,:],m)

Mk = np.zeros(n)
M2 = np.zeros(n*(m+1)).reshape(n,m+1)
for i in range(1,m+1):
    Mk += V_i[:,i] - V_iplus1[:,i-1]
    M2[:,i] = Mk

V0_upperbound2_ = np.max(g(X2.T,np.arange(0,m+1)).T - M2.T, axis=0)
V0_upperbound2 = np.mean(V0_upperbound2_)
std_upperbound2 = np.std(V0_upperbound2_)
conv_interval_upperbound2 = V0_upperbound2 + np.array([-1,1]) * 1.96 * std_upperbound2 / np.sqrt(n)
final_upperbound2 = conv_interval_upperbound2[1]

print("Estimator:", V0_upperbound2)
print("Standard deviation:", std_upperbound2 / np.sqrt(n))
print("Condidence interval 95%:", conv_interval_upperbound2)
print("Error:", 100 * 1.96 * std_upperbound2 / (V0_upperbound2 * np.sqrt(n)), "%")

100%|██████████| 10000/10000 [00:27<00:00, 369.44it/s]


Estimator: 15.0647911656221
Standard deviation: 0.007595253762545056
Condidence interval 95%: [15.04990447 15.07967786]
Error: 0.09881781440528561 %


In [200]:
print("Confidence interval (lower & upper bounds):", [final_lowerbound, final_upperbound2])

Confidence interval (lower & upper bounds): [14.547926252324377, 15.07967786299669]


In [201]:
# to observe what base functions have the higher weights
print(alpha_tau)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-3.75443870e+03  1.01446882e+02 -9.06837732e-01  2.68874235e-03
   2.25787441e-01  8.61117202e-02  2.15850481e-03]
 [ 2.74994208e+02 -5.65490920e+00  4.02363394e-02 -9.79851716e-05
   2.20246929e-02 -8.42210993e-04 -1.67831136e-04]
 [ 1.57025307e+02 -2.80202007e+00  1.72054805e-02 -3.62294348e-05
   8.15743412e-02  3.32385360e-03 -9.19554008e-05]
 [ 2.39477199e+02 -4.63704431e+00  3.05371225e-02 -6.80382283e-05
  -1.06381037e-01  5.01129689e-03 -1.83869015e-04]
 [ 8.02068457e+01 -1.09391303e+00  4.41315190e-03 -4.37187111e-06
   2.02997654e-01  5.66067171e-03 -5.44974077e-05]
 [ 1.63534710e+02 -2.94523658e+00  1.77880577e-02 -3.59337796e-05
  -5.39441401e-02  1.20445977e-02 -2.18226546e-04]
 [ 1.34026293e+02 -2.30983834e+00  1.31493285e-02 -2.47033224e-05
   8.53764072e-02  7.81163902e-03 -1.31483425e-04]
 [ 1.39842149e+02 -2.47811455e+00  1.44562830e-02 -2.77153452e-0

# Finite difference